## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/train.csv', engine="python", encoding='latin-1', error_bad_lines=False)

<ipython-input-2-9fd9b59d3410>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('/content/train.csv', engine="python", encoding='latin-1', error_bad_lines=False)
Skipping line 2826: unexpected end of data


In [3]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.drop('id',axis=1,inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2824 entries, 0 to 2823
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2748 non-null   object
 1   author  2540 non-null   object
 2   text    2817 non-null   object
 3   label   2824 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 88.4+ KB


In [7]:
df.shape

(2824, 4)

In [8]:
df.head()

,title,author,text,label
0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.isnull().sum()

title      76
author    284
text        7
label       0
dtype: int64

In [10]:
###Drop Nan Values
df=df.dropna()


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2464 entries, 0 to 2823
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2464 non-null   object
 1   author  2464 non-null   object
 2   text    2464 non-null   object
 3   label   2464 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 96.2+ KB


In [12]:
df.isnull().sum()

title     0
author    0
text      0
label     0
dtype: int64

In [13]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [14]:
## Get the Dependent features
y=df['label']

In [15]:
X.shape

(2464, 3)

In [16]:
y.shape

(2464,)

In [18]:
df.label.value_counts()

0    1388
1    1076
Name: label, dtype: int64

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'2.12.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [22]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [23]:
messages=X.copy()

In [24]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [25]:
messages

,title,author,text
0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
2819,ISIS Chainsaw Massacre: Nine Christian Youths ...,admin,Ads ISIS Chainsaw Massacre: Nine Christian You...
2820,Would a Trump Presidency Mean Economic Disaste...,Neil Irwin,If Donald Trump were elected president and put...
2821,Trump to CNN: âYou Are Fake Newsâ - Breitbart,Ian Hanchett,"During a press conference on Wednesday, Dona..."
2822,Monsanto Weed Killer Roundup Faces New Doubts ...,Danny Hakim,"The reputation of Roundup, whose active ingred..."


In [26]:
messages.reset_index(inplace=True)

In [27]:
messages

,index,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
2459,2819,ISIS Chainsaw Massacre: Nine Christian Youths ...,admin,Ads ISIS Chainsaw Massacre: Nine Christian You...
2460,2820,Would a Trump Presidency Mean Economic Disaste...,Neil Irwin,If Donald Trump were elected president and put...
2461,2821,Trump to CNN: âYou Are Fake Newsâ - Breitbart,Ian Hanchett,"During a press conference on Wednesday, Dona..."
2462,2822,Monsanto Weed Killer Roundup Faces New Doubts ...,Danny Hakim,"The reputation of Roundup, whose active ingred..."


In [28]:
import nltk
import re
from nltk.corpus import stopwords

In [29]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay 2 5 million releas everyon arrest dakota access pipelin',
 'fbi clos

In [31]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [32]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[790, 4217, 3512, 2919, 1616, 2124, 2192, 4266, 2856, 4274],
 [2732, 3028, 3815, 172, 1804, 3358, 3903],
 [3118, 1296, 1173, 4832],
 [2539, 4232, 3574, 3922, 4256, 1390],
 [1174, 1804, 3604, 2980, 1934, 3355, 1804, 2246, 3747, 2285],
 [1086,
  1576,
  2078,
  1101,
  3314,
  4336,
  1420,
  3348,
  3794,
  656,
  2379,
  2938,
  4676,
  2646,
  3903],
 [4777, 1634, 3444, 4254, 4393, 2790, 1897, 139, 4627, 892, 974],
 [2490, 3790, 1868, 2150, 3742, 652, 4336, 2297, 4627, 892, 974],
 [1988, 432, 4660, 4298, 1631, 3922, 1349, 2364, 4336, 2382],
 [29, 1638, 1067, 4298, 3637, 3421, 4039, 2186],
 [1364, 234, 3766, 575, 1688, 500, 811, 551, 1728, 3692, 1031],
 [3922, 3911, 1616, 3922, 4336, 3742],
 [1598, 2884, 1483, 2370, 3308, 1429, 4251, 1880, 2143],
 [2176, 2171, 2933, 2817, 2912, 4879, 2198, 4627, 892, 974],
 [4002, 3083, 2941, 2932, 337, 4627, 892, 974],
 [3946, 3711, 534, 4148, 4553, 3656, 1740, 401, 4711, 3199],
 [3339, 3198, 3028],
 [1918, 4899, 908, 3921, 4336, 2113, 2306, 3903],
 

In [33]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [34]:
onehot_repr[1]

[2732, 3028, 3815, 172, 1804, 3358, 3903]

### Embedding Representation

In [35]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 790 4217 3512 ...    0    0    0]
 [2732 3028 3815 ...    0    0    0]
 [3118 1296 1173 ...    0    0    0]
 ...
 [4336 1185 2050 ...    0    0    0]
 [4083 2491 1761 ...    0    0    0]
 [1535  702 2542 ...    0    0    0]]


In [36]:
embedded_docs[1]

array([2732, 3028, 3815,  172, 1804, 3358, 3903,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [37]:
embedded_docs[0]

array([ 790, 4217, 3512, 2919, 1616, 2124, 2192, 4266, 2856, 4274,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [39]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
len(embedded_docs),y.shape

(2464, (2464,))

In [41]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [42]:
X_final.shape,y_final.shape

((2464, 20), (2464,))

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [44]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
26/26 [==============================] - 5s 79ms/step - loss: 0.6628 - accuracy: 0.5697 - val_loss: 0.4800 - val_accuracy: 0.7666
Epoch 2/10
26/26 [==============================] - 1s 54ms/step - loss: 0.3384 - accuracy: 0.8715 - val_loss: 0.3362 - val_accuracy: 0.8563
Epoch 3/10
26/26 [==============================] - 1s 49ms/step - loss: 0.1372 - accuracy: 0.9618 - val_loss: 0.3506 - val_accuracy: 0.8636
Epoch 4/10
26/26 [==============================] - 2s 83ms/step - loss: 0.0428 - accuracy: 0.9891 - val_loss: 0.4589 - val_accuracy: 0.8477
Epoch 5/10
26/26 [==============================] - 1s 56ms/step - loss: 0.0112 - accuracy: 0.9982 - val_loss: 0.5020 - val_accuracy: 0.8526
Epoch 6/10
26/26 [==============================] - 1s 49ms/step - loss: 0.0063 - accuracy: 0.9994 - val_loss: 0.7972 - val_accuracy: 0.8256
Epoch 7/10
26/26 [==============================] - 1s 48ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.6668 - val_accuracy: 0.8428
Epoch 8/10
26

### Adding Dropout

In [45]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [46]:
y_pred=model.predict(X_test)

26/26 [==============================] - 1s 13ms/step


In [47]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [48]:
from sklearn.metrics import confusion_matrix

In [49]:
confusion_matrix(y_test,y_pred)

array([[464,   0],
       [350,   0]])

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5700245700245701

In [51]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.73       464
           1       0.00      0.00      0.00       350

    accuracy                           0.57       814
   macro avg       0.29      0.50      0.36       814
weighted avg       0.32      0.57      0.41       814



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
